In [1]:
from pomegranate import *
from my_bayes import *

In [2]:
burglary = DiscreteDistribution(dict(auto_complete([['b', 0.001]])))
earthquake = DiscreteDistribution(dict(auto_complete([['e', 0.002]])))
alarm = ConditionalProbabilityTable(
    auto_complete([
        ['b', 'e', 'a', 0.95],
        ['b', '-e', 'a', 0.94],
        ['-b', 'e', 'a', 0.29],
        ['-b', '-e', 'a', 0.001]
    ]),
    [burglary, earthquake]
)
john_calls = ConditionalProbabilityTable(
    auto_complete([
        ['a', 'j', 0.90],
        ['-a', 'j', 0.05]
    ]),
    [alarm]
)
mary_calls = ConditionalProbabilityTable(
    auto_complete([
        ['a', 'm', 0.70],
        ['-a', 'm', 0.01]
    ]),
    [alarm]
)

In [3]:
burglary.parameters

[{'b': 0.001, '-b': 0.999}]

In [4]:
earthquake.parameters

[{'e': 0.002, '-e': 0.998}]

In [5]:
alarm.parameters[0]

[['b', 'e', 'a', 0.95],
 ['b', '-e', 'a', 0.94],
 ['-b', 'e', 'a', 0.29],
 ['-b', '-e', 'a', 0.001],
 ['b', 'e', '-a', 0.050000000000000044],
 ['b', '-e', '-a', 0.06000000000000005],
 ['-b', 'e', '-a', 0.71],
 ['-b', '-e', '-a', 0.999]]

In [6]:
john_calls.parameters[0]

[['a', 'j', 0.9],
 ['-a', 'j', 0.05],
 ['a', '-j', 0.09999999999999998],
 ['-a', '-j', 0.95]]

In [7]:
mary_calls.parameters[0]

[['a', 'm', 0.7],
 ['-a', 'm', 0.01],
 ['a', '-m', 0.30000000000000004],
 ['-a', '-m', 0.99]]

In [8]:
model = MyBayesianNetwork('Burglary')
nodes = [burglary, earthquake, alarm, john_calls, mary_calls]
states = [State(node) for node in nodes]
model.add_states(*states)
model.add_edge(states[0], states[2])
model.add_edge(states[1], states[2])
model.add_edge(states[2], states[3])
model.add_edge(states[2], states[4])
model.bake()

In [9]:
model.probability(['b', 'e', 'a', 'j', 'm']) # P(Burglary, Earthquake, Alarm, JohnCalls, MaryCalls)

1.1969999999999995e-06

In [10]:
model.predict_proba([None, None, None, 'j', 'm'])[2].parameters[0]['a']

0.7606920388631567

In [11]:
notb_j_notm = model.sum_probability(['-b', None, None, 'j', '-m'])

In [12]:
notb_j_notm

0.049641657650999996

In [13]:
notb = burglary.probability('-b')

In [14]:
notb_j_notm / notb  # P(JohnCalls, ¬ MaryCalls | ￢Burglary)

0.049691348999999996